# Welcome to the jupyter notebook for Ro-Vibrational Spectrscopy!
### Created Fall 2022: J. A. DePaolo-Boisvert

In [ ]:
# Importing Packages which are like collections of code that groups of people will find useful
# Some packages are widely known and applied while others can be much more niche
import numpy as np
import scipy as sp
import scipy.constants as cons #constants is a submodule of scipy that we want easy access to
import matplotlib as mp
import matplotlib.pyplot as plt
%matplotlib inline

def plot_data(title, xs, ys, xlabel, ylabel):
    plt.clf()
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    _ = plt.plot(xs, ys)
    plt.show()

def plot_data_series(title, xs, ys, xlabel, ylabel, legend):
    plt.clf()
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    assert len(xs) == len(ys)
    for i in range(len(xs)):
        _ = plt.plot(xs[i], ys[i])
    plt.legend(legend)
    plt.show()

In [ ]:
#Defining Constants
h = cons.Planck # Joule-Seconds
kb = cons.Boltzmann # Joule / Kelvin
c = cons.c # Meters / Second
amutokg = 1.66E-27 # Kilograms / Amu
evtojoule = 1.602176634E-19 # Joules / Electron-Volt (Coulombs / ElementaryCharge)

#Define the diatomic molecule here
mass1 = 1.007825 *amutokg
mass2 = 34.968853 *amutokg
#H=1.007825, D=2.014102, Cl(35)=34.968853, and Cl(37)=36.965903

#Define bond strength and temperature
k_force = 481 # Newtons/Meter
temp = 300 # Kelvin

### 'From Physics we know that the vibrational frequency of a harmonic' oscillator is
$f = \frac{1}{2*\pi}*\sqrt{k/\mu}$

In [ ]:
#Reduced Mass
reduced_mass = lambda m1, m2: (m1*m2)/(m1+m2)

#Frequency Function
Freq = lambda k_force, mu: np.sqrt(k_force/mu)/(2*cons.pi)

#Obtain Frequency
freq_vib = Freq(k_force, reduced_mass(mass1, mass2))

#Convert Units
freq_vib_wvnm = freq_vib/(cons.c*100)

print(freq_vib, freq_vib_wvnm)

#### From Equation 1 of the text
The energy levels of a harmonic oscillator are given by the base frequency and the level number.
$$
E(\nu_o, \nu) = h\nu_0(\nu+1/2)
$$


In [ ]:
e_vib_levels = lambda base_freq, level : h*base_freq*(level+0.5)

#Lets say we want to see the frequencies of the first 10 energy levels of this oscillator
E_levels = e_vib_levels(freq_vib, np.arange(0,10)) #Note that this calls for the frequency in Hz

for entry in E_levels:
    print(entry/(cons.h*cons.c*100)) #Convert to wavenumbers

#### From Equation 2 of the text
$$
E(B_e, J) = \frac{h^2}{8 \pi^2 I}J(J+1) = B_eJ(J+1)
$$

In [ ]:
#Calculate the rotational part requires defining the internuclear distance r_nuc
r_nuc = 1.27E-10 #1.27 angstroms
mu = reduced_mass(mass1, mass2)

# Since the reduced mass is known, the moment of inertia is readily calculable
Moment = lambda r_nuc, mu: mu*(r_nuc**2)
mom_I = Moment(r_nuc, mu)

#A function that will obtain the value of the coefficient Be
fB_e = lambda r_nuc : ((cons.h**2)/(8*(cons.pi**2)*mu*(r_nuc**2)))

#A function that will obtain the rotational energy levels
e_rot_levels = lambda B_e, level: B_e*(level*(level+1))

#Obtain the rotational energy levels
E_levels = e_rot_levels(fB_e(r_nuc), np.arange(0,10))

#Now lets see the first 10 rotational energies
for entry in E_levels:
    print(entry/(cons.h*cons.c*100)) #Convert to wavenumbers

#### Based on equation 4 on page 417 of the text, The addition of several correctional factors yields a strong expression for the molecule's physical energy levels
$$
 T(\nu,J) = E_{vib} + E_{rot} + Anharmonicty + Centrifugal + Coupling
$$

$$
 T(\nu,J) = \nu_e(\nu+\frac{1}{2}) + B_eJ(J+1) - \nu_e\chi_e(\nu+\frac{1}{2})^2 - D_eJ^2(J+1)^2 - \alpha_e(\nu+\frac{1}{2})J(J+1)
$$

This equation has a total of 5 parameters, thus

$$
T(\nu,J) = T(\nu,J |\nu_e,\chi_e,B_e,D_e,\alpha_e)
$$

In [ ]:
T_levels = lambda nu, J, nu_e, chi_e, B_e, D_e, a_e : (nu_e*(nu+0.5)) + B_e*J*(J+1) - nu_e*chi_e*(nu+0.5)**2 - D_e * J**2 * (J+1)**2 - a_e*(nu+0.5)*J*(J+1)

In [ ]:
# Retrieve these values for HCl from NIST, and
# generate a matrix of the first 2 vibrational levels, by 25 rotational levels

#Parameters
nu_e = None
chi_e = None
B_e = None
D_e = None
a_e = None

#Number of Levels to use
nu_max = 2
J_max = 25

#Obtain the energies
levels = np.array([[T_levels(i, j, nu_e, chi_e, B_e, D_e, a_e) for j in range(J_max)] for i in range(nu_max)])

In [ ]:
levels

## Boltzmann Distribution
The boltzmann distribution assigns a probability to an energy level based on the temperature:
$$
P_i = e^{\frac{-E_i}{k_b*Temp}}
$$

In [ ]:
boltzmann_prob = lambda energies, temp : np.exp(-1*energies/(cons.Boltzmann*temp))

Populate the energy levels that were previously created with Boltzmann probability.

In [ ]:
#Convert the levels calculated from equation 4 to joules
boltzmann_pop = boltzmann_prob(levels*cons.h*cons.c*100, temp)
print(boltzmann_pop)

### Boltzmann Degeneracy

The rigid rotor model utilizes the spherical harmonics in it's solution - creating degeneracy as the levels increase.\
This is just like the degeneracy of electron orbitals!

In [ ]:
#Degeneracy of Rotational Levels is
rot_degeneracy = lambda J : 2*J + 1

In [ ]:
#Adjust the populations to account for their degeneracies
boltzmann_pop = boltzmann_prob(levels*cons.h*cons.c*100, temp) * rot_degeneracy(np.arange(levels.shape[1]))
print(boltzmann_pop)

In [ ]:
# This can be normalized to sum to one, if desired
normalized_boltzmann_pop = boltzmann_pop/np.sum(boltzmann_pop)
print(normalized_boltzmann_pop, np.sum(normalized_boltzmann_pop))

In [ ]:
print(np.sum(normalized_boltzmann_pop, 1)) #Distribution of vibrational states (summing across rotations))
print(np.sum(normalized_boltzmann_pop, 0)) #Distribution of rotational states (summing across vibrations))

## Selection Rules for spectroscopy
#### Calculating energy levels is good, but the key to spectra is the transitions between energy levels
 Here is a summary of the selection rules: \
$\Delta \nu$ = +/-1 and $\Delta J$ = +/-1 $ \
Any transition must satisfy both of these simultaneously


In [ ]:
levels

In [ ]:
# Typically we are only concerned with vibrational transitions from 0, 1.
r_transitions = levels[1,1:] - levels[0,:-1]
p_transitions = levels[1,:-1] - levels[0,1:]
print(r_transitions)
print(p_transitions)
print(levels[1,0] - levels[0,0])

## Generating A Spectrum

In [ ]:
plt.clf()
plt.scatter(np.arange(J_max), levels[1,:] - levels[0,:])
plt.scatter(np.arange(1, J_max), p_transitions)
plt.scatter(np.arange(J_max -1), r_transitions)
plt.legend(['blue_legend_name',
            'orange_legend_name',
            'green_legend_name'])
plt.xlabel('Base J level')
plt.ylabel('Transition Energy (wnvm)')
plt.title('Figure M')
plt.show()

### Description of Generated Spectrum
A spectrum will be generated as a sum of gaussians centered at each transition wavelength, with relative heights of the boltzmann weight of their base level.
$$
f(λ) = W_b e^\frac{-(λ - λ_0)^2}{2\sigma^2}
$$
With σ as an empirical bandwidth parameter.

In [ ]:
#Define a Gaussian
Gaussian = lambda x, a, b, c : a*np.exp(-1*(x-b)**2 / (2*c**2))

In [ ]:
#Define Coefficients Wb
r_coeffs = normalized_boltzmann_pop[0,:-1] # Values of base levels where dJ += 1 is possible
p_coeffs = normalized_boltzmann_pop[0,1:] # Values of base levels where dJ -= 1 is possible

In [ ]:
r_transitions, r_coeffs

In [ ]:
p_transitions, p_coeffs

In [ ]:
domain = np.linspace(2500,3200,1000)
sigma = 2.5

r_excitation_data = np.array([[Gaussian(domain, W_b, l_c, sigma) for W_b in r_coeffs] for l_c in r_transitions])
p_excitation_data = np.array([[Gaussian(domain, W_b, l_c, sigma) for W_b in p_coeffs] for l_c in p_transitions])

excitation_data = []
for i in range(r_excitation_data.shape[0]):
    excitation_data.append(r_excitation_data[i,i,:])
    excitation_data.append(p_excitation_data[i,i,:])
excitation_data = np.array(excitation_data)

plot_data('TITLE HERE', domain, excitation_data.sum(0), 'Wavelength (wvnm)', 'Qeff')
plot_data('TITLE HERE', domain, 1 - excitation_data.sum(0), 'Wavelength (wvnm)', 'Transmittance')

## Graph Real Data

In [ ]:
#Obtain jcampdx from NIST, a common IR file format at
# https://webbook.nist.gov/cgi/cbook.cgi?JCAMP=C7647010&Index=0&Type=IR
# Save it to your google drive and point to it here
from IPython.testing import test
from google.colab import drive
drive.mount('/content/drive')
##################################################################
hcl_jcamp = '/content/drive/MyDrive/7647-01-0-IR.jdx'
#%cat $hcl_jcamp

In [ ]:
def get_jcamp_data(jcamp_fn):
    with open(jcamp_fn, 'r') as w:
        lines = [line[:-1] if line.endswith('\n') else line for line in w.readlines()]
    #print(lines)
    data_start = lines.index("##XYDATA=(X++(Y..Y))") + 1
    data_end = lines.index("##END=")
    #Data is averaged some number of times
    data_lines = lines[data_start:data_end]
    data = jnp.array([[float(element) for element in line.split(' ')] for line in data_lines])
    x_data = data[:,0]
    y_data = jnp.average(data[:,1:],1)
    return x_data, y_data

In [ ]:
# Obtain X and Y data from the jcamp file
xs_jcamp, ys_jcamp = get_jcamp_data(hcl_jcamp)
# Obtain X and Y data calculate by this notebook
xs_calc, ys_calc = domain, 1 - excitation_data.sum(0)

# Add code between plt.clf() and plt.show() to display the data from NIST
# Use previous plots as an example, make sure there is a title and axis labels
# The two sets of data that you should plot are obtained above

plt.clf()
#your code here
plt.show()

## Questions

### Edit this text cell to write your answers to the following questions below each question.

#### Explain the purpose of prefixes like np. and cons.

#### Explain the meaning/contribution of each term in the equation for T (from equation 4).

#### How does the boltzmann population differ between rotational and vibrational energy levels?  Are either well dispersed/isolated?

#### How do we know if we are using enough boltzmann levels to describe the molecular energy landscape?

#### Describe the selection rules for spectroscopy and the R and P branches.  Which is higher energy?  Why is there a "forbidden transition" between the two branches?

#### Describe and explain Figure M.

#### This is, principally, a single molecule analysis.  What effects or interactions that we have not described here can occur to a molecule present in a bulk/ensemble?

#### How well does this analysis fit/mimick real spectroscopic data?

#### At what pressure of HCl was the spectrum from the NIST link taken (the one we all retrieved together)?  What parameters were used in cell 8 of this notebook (the five parameters retrieved from NIST table)?

#### What (in your opinion) was the best _assumption_ made in this analysis?

#### What (in your opinion) was the worst _assumption_ made in this analysis?


## Assignment

#### Choose a Diatomic Molecule and replicate this analysis
#### No two students may use the same molecule.  Post which molecule you will analyze to the discussion on blackboard.